In [1]:
# -*- coding: utf8 -*-
import codecs
import features
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools as it
import pickle
import datetime

In [2]:
def dateheure():
    return datetime.datetime.utcnow().strftime('%y%m%d%H%M')

typeEchantillon : 
- fixe pour utiliser tailleEchantillon
- variable pour utiliser nombre,increment,nombreInitial
- total pour utiliser tout le lexique

In [20]:
typeEchantillon="fixe"
tailleEchantillon=50000
nombre=1000
increment=250
nombreInitial=20
freqForme=1000000
if typeEchantillon=="total":
    echantillonPrefix="-total"
elif typeEchantillon=="fixe":
    echantillonPrefix="-%d-%s" % (tailleEchantillon,dateheure())
else:
    echantillonPrefix="XXX"

In [4]:
filePrefix="MGC-150811"

##Importations
- codecs pour les encodages
- pandas et numpy pour les calculs sur tableaux
- matplotlib pour les graphiques
- itertools pour les itérateurs sophistiqués (paires sur liste, ...)

In [5]:
#%pylab inline
#pd.options.display.mpl_style = 'default'
debug=False

###Préparation des matrices de traits

features.add_config('bdlexique.ini')
fs=features.FeatureSystem('phonemes')
#fs.supremum.concept.extent

bdlexiqueIn = unicode(u"èò")
bdlexiqueNum = [ord(char) for char in bdlexiqueIn]
neutreOut = unicode(u"EO")
neutralise = dict(zip(bdlexiqueNum, neutreOut))

def recoder(chaine,table=neutralise):
    if type(chaine)==str:
        temp=unicode(chaine.decode('utf8')).translate(table)
        result=temp.encode('utf8')
    elif type(chaine)==unicode:
        result=chaine.translate(table)
    return result

###Définition des cases
- principales pour le paradigme courant
- secondaires pour les cases rares
- totales pour le tout

In [6]:
casesPrincipales= [
        'inf', 'pi1S', 'pi2S', 'pi3S', 'pi1P', 'pi2P', 'pi3P', 'ii1S',
        'ii2S', 'ii3S', 'ii1P', 'ii2P', 'ii3P', 
        'fi1S', 'fi2S', 'fi3S', 'fi1P', 'fi2P',
        'fi3P', 'pI2S', 'pI1P', 'pI2P', 'ps1S', 'ps2S', 'ps3S', 'ps1P',
        'ps2P', 'ps3P', 
        'pc1S', 'pc2S', 'pc3S', 'pc1P', 'pc2P', 'pc3P', 'pP',
        'ppMS', 'ppMP', 'ppFS', 'ppFP'
            ]
casesSecondaires= [
       'ai1S', 'ai2S', 'ai3S', 'ai1P', 'ai2P', 'ai3P', 'is1S', 'is2S', 'is3S', 'is1P', 'is2P', 'is3P'
            ]
casesTotales=casesPrincipales+casesSecondaires
listeCases=casesTotales

In [7]:
with open('MGC-150809-Verbes.pkl', 'rb') as input:
    verbes = pickle.load(input)

verbes["phono"]=verbes["phono"].apply(lambda x: recoder(x))

#Echantillonage

##Assembler les échantillons correspondant à une étape n

In [8]:
def assemblerExtrait(nombre):
    return verbes.ix[np.sort(np.concatenate(tirages[0:nombre]))]

###Tirage incrémental des formes disponibles
- increment : taille de chaque tirage
- nombre : nombre de tirages

np.random.choice donne une liste d'index de formes tirées dans l'ordre du tirage

on découpe la liste en morceaux de la taille de l'incrément

In [21]:
if typeEchantillon=="variable":
    tirage=np.random.choice(verbes.index,size=nombre*increment,p=verbes["prob"],replace=False)
    tirages=[tirage[increment*x:increment*(x+1)] for x in range(len(tirage)/increment+(len(tirage)%increment!=0))]
    extrait=assemblerExtrait(nombreInitial)
elif typeEchantillon=="fixe":
    tirage=np.random.choice(verbes.index,size=tailleEchantillon,p=verbes["prob"],replace=False)
    extrait=verbes.ix[np.sort(tirage)]
elif typeEchantillon=="total":
    extrait=verbes

In [22]:
paradigmes=pd.pivot_table(extrait, values='phono', index=['lexeme'], columns=['case'], aggfunc=lambda x: ",".join(x)).reset_index().reindex()

In [11]:
paradigmes.to_csv(path_or_buf=filePrefix+echantillonPrefix+"-paradigmes.csv",encoding="utf8",sep=";")

In [39]:
#with open(filePrefix+echantillonPrefix+'-Paradigmes.pkl', 'wb') as output:
#   pickle.dump(paradigmes, output, pickle.HIGHEST_PROTOCOL)

In [23]:
extrait

,ortho,phono,ext,cs,ms,vs,lexeme,freq,prob,case
40,a,a,NaN,V,3S,pi,avoir,7.167730e+11,1.479124e-02,pi3S
45,abaissa,abèsa,NaN,V,3S,ai,abaisser,2.640000e+08,5.447871e-06,ai3S
46,abaissai,abèsè,NaN,V,1S,ai,abaisser,1.700000e+07,3.508099e-07,ai1S
47,abaissaient,abèsè,"t""",V,3P,ii,abaisser,4.100000e+07,8.460709e-07,ii3P
50,abaissait,abèsè,"t""",V,3S,ii,abaisser,2.520000e+08,5.200241e-06,ii3S
52,abaissant,abèsâ,"t""",V,,pP,abaisser,2.510000e+08,5.179605e-06,pP
64,abaisse,abEs,@,V,1S,pi,abaisser,4.730000e+08,9.760770e-06,pi1S
66,abaisse,abEs,@,V,2S,pI,abaisser,4.730000e+08,9.760770e-06,pI2S
67,abaisse,abEs,@,V,3S,pi,abaisser,4.730000e+08,9.760770e-06,pi3S
71,abaissent,abEs,"@t""",V,3P,pi,abaisser,1.000000e+08,2.063588e-06,pi3P


In [25]:
extrait[extrait["vs"]=="pp"]

,ortho,phono,ext,cs,ms,vs,lexeme,freq,prob,case
7569,adressées,adrèse,"z""",K,FP,pp,adresser,12875,2.656869e-10,ppFP
14017,allée,ale,NaN,K,FS,pp,aller,1284770,2.651236e-08,ppFS
29171,attendue,atâdy,NaN,K,FS,pp,attendre,205815,4.247173e-09,ppFS
65652,cherchée,SErSe,NaN,K,FS,pp,chercher,116145,2.396754e-09,ppFS
73579,coincés,kwêse,"z""",K,MP,pp,coincer,8756,1.806877e-10,ppMP
80773,connu,kòny,NaN,K,MS,pp,connaître,159029,3.281703e-09,ppMS
96950,demandé,d6mâde,NaN,K,MS,pp,demander,189423,3.908910e-09,ppMS
102270,disparu,dispary,NaN,K,MS,pp,disparaître,34944,7.211001e-10,ppMS
103817,dites,dit,"@z""",K,FP,pp,dire,1077865,2.224269e-08,ppFP
103828,dits,di,"z""",K,MP,pp,dire,1077865,2.224269e-08,ppMP
